In [1]:
# https://stackoverflow.com/questions/66116155/how-to-tell-pytorch-which-cuda-version-to-take
# https://github.com/pytorch/pytorch/issues/75992

# Previous Versions of Pytorch avaiable here: https://pytorch.org/get-started/previous-versions/
#! pip install --upgrade torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
#! pip install --upgrade torch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0
#! pip install --upgrade torch==1.12.1 --extra-index-url https://download.pytorch.org/whl/cu113 #<--This version works for CUDA 11.4



In [2]:
import pandas as pd
import os
import time
import numpy as np
import torch
import gc


In [3]:
import seaborn as sns
from scipy import stats
from statistics import NormalDist
import matplotlib.pyplot as plt

In [4]:
torch.__version__
#'1.12.0+cu111'
#'1.9.0+cu111' This is the version by default


'1.12.1+cu113'

In [5]:
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('CPU')
physical_devices = tf.config.list_physical_devices('GPU')

In [6]:
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device
device = 'cpu'


In [8]:
#test
x = torch.rand(5, 3, 
               device=device#'cpu'
               )
print(x)

tensor([[0.2082, 0.3773, 0.5927],
        [0.2154, 0.9319, 0.1089],
        [0.3683, 0.1989, 0.8518],
        [0.0697, 0.6043, 0.5073],
        [0.0594, 0.7900, 0.2637]])


In [9]:
x.get_device()

-1

In [10]:
#Deleting tensor to free memory
del x
torch.cuda.empty_cache()

In [11]:
torch.cuda.memory_allocated()

0

In [12]:
#! conda install pytorch torchvision torchaudio cudatoolkit=10.1 -c pytorch
#! pip3 install torch==1.7.0 torchvision==0.8.1 -f https://download.pytorch.org/whl/cu101/torch_stable.html
#! pip install tensorflow
! nvidia-smi

Mon Feb 20 20:33:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   30C    P0    35W / 250W |      3MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Logits Extractor
>
> Extracting Tensor Logits from a given Neural Code Model @danaderp
>

In [13]:
#Available Datasets
# Case1: codesearch_tesbed_EleutherAI-gpt-neo-125M_10000 for the model 'EleutherAI/gpt-neo-125M' 
# Case2: codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000 for the model 'EleutherAI/gpt-neo-2.7B' <-- MEMORY CONSTRAINTS

def params(): 
    
    code_models = {
        'Case3':('EleutherAI/gpt-neo-1.3B','codesearch_tesbed_EleutherAI-gpt-neo-1.3B_10000.csv','EleutherAI-gpt-neo-1.3B_10000_','callbacks-EleutherAI-gpt-neo-1.3B_10000_'),
        'Case4':('microsoft/CodeGPT-small-py','codesearch_tesbed_microsoft-CodeGPT-small-py_1024_10000.csv','CodeGPT-small-py_10000_','callbacks-CodeGPT-small-py_10000_'),
        'Case5':('microsoft/CodeGPT-small-py-adaptedGPT2','codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_1024_10000.csv','CodeGPT-small-py-adaptedGPT2_10000_','callbacks-CodeGPT-small-py-adaptedGPT2_10000_'),
        'Case6':('Salesforce/codegen-2B-multi','codesearch_tesbed_Salesforce-codegen-2B-multi_10000.csv','Salesforce-codegen-2B-multi_10000_','callbacks-Salesforce-codegen-2B-multi_10000_')
    }
    current_case = 'Case5' #<----[Hyper]
    
    #print(code_models[current_case][1])
    
    return {
            'big_table_path' : '../data/concept_tables/' + code_models[current_case][1],
            'hf_model' :  code_models[current_case][0],
            'model_name': code_models[current_case][2],
            'callbacks' : '../data/' + code_models[current_case][3],
            'wpe':1024  #<----[Hyper]
}

In [14]:
#pwd
parameters = params()
parameters['big_table_path']

'../data/concept_tables/codesearch_tesbed_Salesforce-codegen-2B-multi_10000.csv'

# Data Upload

In [15]:
data_pd = pd.read_csv( 
                      parameters['big_table_path'] , 
                      index_col=0
            )

In [16]:
data_pd.describe()

,model_total_input_ids
count,10000.000000
mean,285.659700
std,278.421629
min,35.000000
25%,114.000000
50%,191.000000
75%,346.250000
max,2043.000000


In [17]:
data_pd.head(5)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids
0,"def build_top_graph(self,tokens):\n """"""...","[('def', 'def', 'function_definition'), ('buil...","[(4299, 'def', 'function_definition'), (1382, ...","[4299, 1382, 62, 4852, 62, 34960, 7, 944, 11, ...",124
1,"def add_linguistic_type(self, lingtype, constr...","[('def', 'def', 'function_definition'), ('add_...","[(4299, 'def', 'function_definition'), (751, '...","[4299, 751, 62, 1359, 84, 2569, 62, 4906, 7, 9...",353
2,"def layers(self):\n """"""Yield a shallow ...","[('def', 'def', 'function_definition'), ('laye...","[(4299, 'def', 'function_definition'), (11685,...","[4299, 11685, 7, 944, 2599, 198, 50280, 37811,...",817
3,"def list_tags(self):\n """"""\n Lis...","[('def', 'def', 'function_definition'), ('list...","[(4299, 'def', 'function_definition'), (1351, ...","[4299, 1351, 62, 31499, 7, 944, 2599, 198, 502...",194
4,"def add_column_healpix(self, name=""healpix"", l...","[('def', 'def', 'function_definition'), ('add_...","[(4299, 'def', 'function_definition'), (751, '...","[4299, 751, 62, 28665, 62, 258, 282, 79, 844, ...",307


# Model Upload

In [18]:
#tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
#generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")
#pip list
#! pip install git+https://github.com/huggingface/transfomers.git

def testing_1():
    #! pip install transformers
    from transformers import AutoTokenizer, AutoModelForCausalLM
    from transformers import pipeline
    # This is for 'EleutherAI/gpt-neo-125M'
    tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
    generator = pipeline(
        'text-generation', 
        model= parameters['hf_model'] )
    
    #TEST: Example 1: generation
    generator(
        "EleutherAI has", 
        do_sample=True, 
        max_new_tokens=20, 
        pad_token_id = tokenizer.eos_token_id
        )
    
    #TEST: Example 2: generation

    generator( 
        data_pd.whole_func_string.values[0][:100], #<-- Code data
        do_sample=True, 
        max_new_tokens=20,
        pad_token_id = tokenizer.eos_token_id 
        )



# Extracting Logits From a Given Model

In [19]:
#This code works for GPT-Neo
#from transformers import GPTNeoForCausalLM, GPT2Tokenizer
#model = GPTNeoForCausalLM.from_pretrained( parameters['hf_model'] )
#tokenizer = GPT2Tokenizer.from_pretrained( parameters['hf_model'] )

In [20]:
parameters['hf_model']

'Salesforce/codegen-2B-multi'

In [21]:
#This code works for CodeGPT and Salesforce
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
model = AutoModelForCausalLM.from_pretrained(parameters['hf_model'])


In [22]:
#tokenizer.get_vocab()
#len( tokenizer.get_vocab() ) #todo an assert

In [23]:
#prompts = data_pd.whole_func_string.values[:2]
prompts = data_pd.whole_func_string.values

In [24]:
len(prompts)

10000

In [25]:
#Making space for the tensors
#input_ids_list = tokenizer.batch_encode_plus( prompts ) #<-- Do not return as a Tensor
input_ids_list = tokenizer.batch_encode_plus( list(prompts) ) #<-- Do not return as a Tensor [cast to list]
#input_ids_list = tokenizer.batch_encode_plus( prompts , return_tensors="pt") #<-- "pt" returns as a Tensor

In [26]:
#Casting Integers to Tensor Integers. Make sure the tesor is created in a device
#We ignored the parameter attention_mask since we are not using masking here [https://huggingface.co/transformers/v4.10.1/glossary.html#attention-mask]

#input_ids_list = [torch.Tensor( np.array( input_ids ) ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
#input_ids_list = [ input_ids for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
input_ids_list = [torch.tensor(  input_ids, dtype = torch.int, device=device ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]

In [27]:
input_ids_list[0]

tensor([ 4299,  1382,    62,  4852,    62, 34960,     7,   944,    11,    83,
          482,   641,  2599,   198, 50280, 37811, 10934,   257,  1793,   313,
         4823,  4554,   422, 44267,  1366,    13,   198, 50280, 37811,   198,
        50280,     2,  3497,  4096,  4823,  1321,    13,   198, 50280,   301,
         2012,   796, 16326,    58,    15,    60,  6624,   705,   301,  2012,
            6,   198, 50280, 34960,  4906,   796, 16326,    58,    16,    60,
          198, 50280, 34762,   796,  4823,  4906,  6624,   705, 12894,  1470,
            6,   198, 50280, 34960,  3672,   796, 16326,    58,    17,    60,
          198, 50280,     2, 10934,   262,  4823,   198, 50280, 34960,   796,
        29681,     7,  2389,    28, 34960,  3672,    11,  7646,    28,   301,
         2012,    11,  7924,    28, 34762,     8,   198, 50280,   944,    13,
        34960,   796,  2116,    13, 11249,    62, 34960,     7, 34960,    11,
        16326,    58,    18, 12962], dtype=torch.int32)

In [28]:
#It should be same size
print(len(input_ids_list),len(prompts))
assert len(input_ids_list) == len(prompts)

10000 10000


In [29]:
data_pd.model_input_ids.values[0]

'[4299, 1382, 62, 4852, 62, 34960, 7, 944, 11, 83, 482, 641, 2599, 198, 50280, 37811, 10934, 257, 1793, 313, 4823, 4554, 422, 44267, 1366, 13, 198, 50280, 37811, 198, 50280, 2, 3497, 4096, 4823, 1321, 13, 198, 50280, 301, 2012, 796, 16326, 58, 15, 60, 6624, 705, 301, 2012, 6, 198, 50280, 34960, 4906, 796, 16326, 58, 16, 60, 198, 50280, 34762, 796, 4823, 4906, 6624, 705, 12894, 1470, 6, 198, 50280, 34960, 3672, 796, 16326, 58, 17, 60, 198, 50280, 2, 10934, 262, 4823, 198, 50280, 34960, 796, 29681, 7, 2389, 28, 34960, 3672, 11, 7646, 28, 301, 2012, 11, 7924, 28, 34762, 8, 198, 50280, 944, 13, 34960, 796, 2116, 13, 11249, 62, 34960, 7, 34960, 11, 16326, 58, 18, 12962]'

In [30]:
input_ids_list[0]

tensor([ 4299,  1382,    62,  4852,    62, 34960,     7,   944,    11,    83,
          482,   641,  2599,   198, 50280, 37811, 10934,   257,  1793,   313,
         4823,  4554,   422, 44267,  1366,    13,   198, 50280, 37811,   198,
        50280,     2,  3497,  4096,  4823,  1321,    13,   198, 50280,   301,
         2012,   796, 16326,    58,    15,    60,  6624,   705,   301,  2012,
            6,   198, 50280, 34960,  4906,   796, 16326,    58,    16,    60,
          198, 50280, 34762,   796,  4823,  4906,  6624,   705, 12894,  1470,
            6,   198, 50280, 34960,  3672,   796, 16326,    58,    17,    60,
          198, 50280,     2, 10934,   262,  4823,   198, 50280, 34960,   796,
        29681,     7,  2389,    28, 34960,  3672,    11,  7646,    28,   301,
         2012,    11,  7924,    28, 34762,     8,   198, 50280,   944,    13,
        34960,   796,  2116,    13, 11249,    62, 34960,     7, 34960,    11,
        16326,    58,    18, 12962], dtype=torch.int32)

In [31]:
# tokenizer.decode( input_ids_list[0] ) #Decoding IDs

In [32]:
#model.to( 'cpu' ) #WARNING, 
model.to( device ) #WARNING, Verify the device before assigning to memory

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 2560)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): CodeGenBlock(
        (ln_1): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear(in_features=2560, out_features=7680, bias=False)
          (out_proj): Linear(in_features=2560, out_features=2560, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear(in_features=2560, out_features=10240, bias=True)
          (fc_out): Linear(in_features=10240, out_features=2560, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): CodeGenBlock(
        (ln_1): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dro

In [33]:
parameters #Verification Point of Parameters

{'big_table_path': '../data/concept_tables/codesearch_tesbed_Salesforce-codegen-2B-multi_10000.csv',
 'hf_model': 'Salesforce/codegen-2B-multi',
 'model_name': 'Salesforce-codegen-2B-multi_10000_',
 'callbacks': '../data/callbacks-Salesforce-codegen-2B-multi_10000_'}

In [34]:
def logit_extractor(batch, input, from_index=0):
    """
    Output is the class CausalLMOutputWithPast (https://huggingface.co/transformers/v4.10.1/main_classes/output.html?highlight=causallmoutputwithpast)"
    logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size)) – Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
    The expression i.type(torch.LongTensor).to(device) is for casting labels for the loss
    """
    #Output is in CausalLMOutputWithPast

    for idx, n in enumerate( range( from_index, len(input), batch) ):
        
        #output = [ model( 
        #    input_ids = i, 
        #    labels = i.type(torch.LongTensor).to(device) 
        #    ) for i in input[n:n+batch] ] #Labels must be provided to compute loss
        output = []
        for i in input[n:n+batch]:
            #print(i)
            output.append( 
                model( 
                    input_ids = i, 
                    labels = i.type(torch.LongTensor).to(device) 
                )
            )
        
        #print(output)
        
        output_logits = [ o.logits.detach().to('cpu').numpy() for o in output ]  #Logits Extraction
        output_loss = np.array([ o.loss.detach().to('cpu').numpy() for o in output ])  #Language modeling loss (for next-token prediction).

        #Saving Callbacks
        current_batch = idx + (from_index//batch)
        for jdx, o_logits in enumerate( output_logits ):
            np.save( parameters['callbacks']+ '/'+parameters['model_name']  + f'_logits_tensor[{jdx+n}]_batch[{current_batch}].npy', o_logits)
        np.save( parameters['callbacks']+ '/'+parameters['model_name']+f'_loss_batch[{current_batch}].npy', output_loss)
        
        print(f"Batch [{current_batch}] Completed")
        
        #Memory Released
        for out in output:
            del out.logits
            torch.cuda.empty_cache()
            del out.loss
            torch.cuda.empty_cache()
        for out in output_logits:
            del out
            torch.cuda.empty_cache()
        for out in output_loss:
            del out
            torch.cuda.empty_cache()
            

    pass

In [35]:
## ACTUAL EXPERIMENT
## TIME AND MEMORY CONSUMING
logit_extractor(batch = 1, input= input_ids_list, from_index=22)

/usr/local/lib/python3.8/dist-packages/transformers/models/codegen/modeling_codegen.py:166: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  ../aten/src/ATen/native/TensorCompare.cpp:402.)
  attn_weights = torch.where(causal_mask, attn_weights, mask_value)


Batch [22] Completed
Batch [23] Completed
Batch [24] Completed
Batch [25] Completed
Batch [26] Completed
Batch [27] Completed
Batch [28] Completed
Batch [29] Completed
Batch [30] Completed
Batch [31] Completed
Batch [32] Completed
Batch [33] Completed
Batch [34] Completed
Batch [35] Completed
Batch [36] Completed
Batch [37] Completed
Batch [38] Completed
Batch [39] Completed
Batch [40] Completed
Batch [41] Completed
Batch [42] Completed
Batch [43] Completed
Batch [44] Completed
Batch [45] Completed
Batch [46] Completed
Batch [47] Completed
Batch [48] Completed
Batch [49] Completed
Batch [50] Completed
Batch [51] Completed
Batch [52] Completed
Batch [53] Completed
Batch [54] Completed
Batch [55] Completed
Batch [56] Completed
Batch [57] Completed
Batch [58] Completed
Batch [59] Completed
Batch [60] Completed
Batch [61] Completed
Batch [62] Completed
Batch [63] Completed
Batch [64] Completed
Batch [65] Completed
Batch [66] Completed
Batch [67] Completed
Batch [68] Completed
Batch [69] Co

In [ ]:
#logit_extractor(batch =2, input= input_ids_list[:2]) #<---- [WARNING TIME AND MEMORY CONSUMING]

In [ ]:
#output_logits = np.load('../data/callbacks/logits_tensor[0]_batch[0].npy')

In [ ]:
#assert output_logits.shape[0] == len(input_ids_list[0])

In [ ]:
#output_loss = np.load('../data/callbacks/loss_batch[0].npy')

In [ ]:
#output_loss